In [ ]:

## This notebook uses Sales, Revenue data to report Sales by different chartfields.

    Sales by region
    Sales by Period
    Sales by Vendor
    Sales by Item
    Sales by Item Category

In [1]:
using Pkg
Pkg.add("DataFrames")
Pkg.add("Dates")
Pkg.add("CategoricalArrays")
Pkg.add("Interact")
Pkg.add("WebIO")
Pkg.add("CSV")
Pkg.add("XLSX")
Pkg.add("DelimitedFiles")
Pkg.add("Distributions")
Pkg.status();

      Status `~/amit/WIP/eCommerce.ai/Project.toml`
  [336ed68f] CSV v0.10.4
  [13f3f980] CairoMakie v0.7.5
  [324d7699] CategoricalArrays v0.10.5
  [a93c6f00] DataFrames v1.3.4
  [31c24e10] Distributions v0.25.58
  [e30172f5] Documenter v0.27.17
  [c601a237] Interact v0.10.4
  [91a5bcdd] Plots v1.29.0
  [0f1e0344] WebIO v0.8.17
  [fdbf4ff8] XLSX v0.7.10
  [ade2ca70] Dates
  [8bb1440f] DelimitedFiles


    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `~/amit/WIP/eCommerce.ai/Project.toml`
  No Changes to `~/amit/WIP/eCommerce.ai/Manifest.toml`
   Resolving package versions...
    Updating `~/amit/WIP/eCommerce.ai/Project.toml`
  [ade2ca70] + Dates
  No Changes to `~/amit/WIP/eCommerce.ai/Manifest.toml`
   Resolving package versions...
    Updating `~/amit/WIP/eCommerce.ai/Project.toml`
  [324d7699] + CategoricalArrays v0.10.5
    Updating `~/amit/WIP/eCommerce.ai/Manifest.toml`
  [324d7699] + CategoricalArrays v0.10.5
   Resolving package versions...
   Installed ImageIO ──────────── v0.6.5
   Installed GR_jll ───────────── v0.64.3+0
   Installed EllipsisNotation ─── v1.6.0
   Installed PDMats ───────────── v0.11.10
   Installed Static ───────────── v0.6.5
   Installed OffsetArrays ─────── v1.11.2
   Installed SpecialFunctions ─── v2.1.5
   Installed ArrayInte

In [ ]:
# Item Master

###############################
## create SUPPLY CHAIN DATA ###
###############################
# Item master, Item Attribs, Item Costing ##
#       UNSPSC, GUDID, GTIN, GMDN
############################################

##########
# UNSPSC #
##########
# UNSPSC file can be downloaded from this link https://www.ungm.org/Public/UNSPSC
xf = XLSX.readxlsx("../sampleData/UNGM_UNSPSC_09-Apr-2022..xlsx")
# xf will display names of sheets and rows with data
# let's read this data in to a DataFrame

# using below command will read xlsx data into DataFrame but will not render column labels
# df = DataFrame(XLSX.readdata("UNGM_UNSPSC_09-Apr-2022..xlsx", "UNSPSC", "A1:D12988"), :auto)
dfUNSPSC = DataFrame(XLSX.readtable("sampleData/UNGM_UNSPSC_09-Apr-2022..xlsx", "UNSPSC")...)
# ... operator will splat the tuple (data, column_labels) into the constructor of DataFrame

# replace missing values with an integer 99999
replace!(dfUNSPSC."Parent key", missing => 99999)
size(dfUNSPSC)

# let's export this clean csv, we'll load this into database
# CSV.write("UNSPSC.csv", dfUNSPSC)

# # remember to empty dataFrame after usage
# # Julia will flush it out automatically after session,
# # but often ERP data gets bulky during session
# Base.summarysize(dfUNSPSC)
# empty!(dfUNSPSC)
# Base.summarysize(dfUNSPSC)

In [ ]:
## devices (products)

## device.txt is 1GB dateset , so please download it from Internet first
data, header = readdlm("../sampleData/GUDID/device.txt", '|', header=true)
dfGUDIDdevice = DataFrame(data, vec(header))

## get vendor GTIN list
dfGTIN = DataFrame(XLSX.readtable("../sampleData/DS_GTIN_ALL.xlsx", "Worksheet";first_row=14)...)

In [ ]:
## now create vendor master from device, contract and GTIN files

#################
# Vendor master #
#################
# create Vendor Master from GUDID dataset
# show(first(dfGUDIDdevice,5), allcols=true)
# show(first(dfGUDIDdevice[:,[:brandName, :catalogNumber, :dunsNumber, :companyName, :rx, :otc]],5), allcols=true)
# names(dfGUDIDdevice)
# dfVendor = unique(dfGUDIDdevice[:,[:brandName, :catalogNumber, :dunsNumber, :companyName, :rx, :otc]])
# dfVendor = unique(dfGUDIDdevice[:,[:companyName]]) # 7574 unique vendors
dfVendor = unique(dfGUDIDdevice[:,[:brandName, :dunsNumber, :companyName, :rx, :otc]])
# dfVendor is a good dataset, have 216k rows for 7574 unique vendors

# # remember to empty dataFrame after usage
# # Julia will flush it out automatically after session,
# # but often ERP data gets bulky during session
# Base.summarysize(dfVendor)
# empty!(dfVendor)
# Base.summarysize(dfVendor)

# CSV.write("VENDOR.csv", dfVendor[1:1000,:])

In [ ]:
# LOCATION Master

data, header = readdlm("../sampleData/uscities.csv", ',', header=true)
dfLocation = DataFrame(data, vec(header))

# # remember to empty dataFrame after usage
# # Julia will flush it out automatically after session,
# # but often ERP data gets bulky during session
# Base.summarysize(dfLocation)
# empty!(dfLocation)
# Base.summarysize(dfLocation)

# CSV.write("LOCATION_MASTER.csv", dfLocation[1:1000,:])

In [ ]:
# Org Master
dfOrgMaster = DataFrame(
    ENTITY=repeat(["HeadOffice"], inner=8),
    GROUP=repeat(["Operations"], inner=8),
    DEPARTMENT=["Procurement","Procurement","Procurement","Procurement","Procurement","HR","HR","MFG"],
    UNIT=["Sourcing","Sourcing","Maintenance","Support","Services","Helpdesk","ServiceCall","IT"])

    # CSV.write("ORG_MASTER.csv", dfOrgMaster[:,:])

In [ ]:
## THIS IS FINAL REVENUE/SALES Register, prepared data for self Service / adhoc analytics

sampleSize = 1000 # number of rows, scale as needed

dfREVENUE = DataFrame(
    UNIT = rand(dfOrgMaster.UNIT, sampleSize),
    SALES_DATE=rand(collect(Date(2020,1,1):Day(1):Date(2022,5,1)), sampleSize),
    STATUS=rand(["Sold","Pending","Hold","Cancelled","Exception"], sampleSize),
    SALES_RECEIPT_NUM = rand(10001:9999999, sampleSize),
    CUSTOMER=rand(unique(dfVendor.companyName), sampleSize),
    GUDID = rand(dfGUDIDdevice.PrimaryDI, sampleSize),
    QTY = rand(1:150, sampleSize),
    UNIT_PRICE = rand(Normal(100, 2), sampleSize)
    );
show(first(dfREVENUE, 5),allcols=true)
